## 第三部分：动作识别实际运行部分（小程序使用时在服务器上运行）
本部分采用flask框架，作用为：将小程序实时发送的六轴原始数据进行数据预处理和特征提取等操作，并将处理好的特征数据投入预先训练好的模型中，得到模型分类结果，最后将模型分类结果返回到小程序端。
本部分依然对IOS和安卓进行差别处理，具体方法见后面的注释。

In [ ]:
import scipy as sp
import sys
from scipy import signal
from scipy import stats
from scipy.stats import entropy
from scipy.stats import iqr as IQR
import joblib
from detecta import detect_peaks

import numpy as np
from flask import Flask,request
app = Flask(__name__)

accx = []
accy = []
accz = []
gryx = []
gryy = []
gryz = []
feature = []


### 定义特征提取函数（同第一部分：动作识别数据处理部分）

In [ ]:
def mean(inArray):
    array=np.array(inArray)
    mean_value = float(array.mean())
    return mean_value

# std: standard deviation of mag column
def std(inArray):
    array=np.array(inArray)
    std_value = float(array.std()) # std value
    return std_value

# mad: median deviation
def mad(inArray):
    array=np.array(inArray)
    mad_value = float(sp.median(array))# median deviation value of mag_column
    return mad_value

# max
def max(inArray):
    array=np.array(inArray)
    max_value=float(array.max()) # max value
    return max_value
# min
def min(inArray):
    array=np.array(inArray)
    min_value= float(array.min()) # min value
    return min_value

# IQR
def IQR(inArray):
    array=np.array(inArray)
    IQR_value=float(IQR(array))# Q3(column)-Q1(column)
    return IQR_value

# Entropy
def entropy(inArray):
    array=np.array(inArray)
    entropy_value=float(entropy(array)) # entropy signal
    return entropy_value

def sma(inArray):
    array=np.array(inArray)
    sma_axial=float(abs(array).sum()) # sum of areas under each signal
    return sma_axial # return sma value

def energy(inArray):
    array=np.array(inArray)
    energy_vector=(array**2).sum() # energy value of each df column
    return energy_vector # return energy vector energy_X,energy_Y,energy_Z

def skew(inArray):
    array=np.array(inArray)
    skew_value=float(stats.skew(array)) # entropy signal
    return skew_value

def kurt(inArray):
    array=np.array(inArray)
    kurt_value=float(stats.kurtosis(array)) # entropy signal
    return kurt_value

### 波峰计数函数

In [ ]:
@app.route('/count', methods=['post'])
def count():

    numA = 0
    numB = 0
    numC = 0
    numD = 0
    count = []

#### 接收小程序端终止识别后的整个accy数组

In [ ]:
    data = request.get_json()
    system = data['system']
    accyAll = data['accyAll']
    activity = data['activity']
    activity.append(5)   #在尾部补充一个label 5（站立），为了避免因六轴数据截取不完整而导致的动作误判，将其默认为站立（无动作）
    if len(accyAll) > 0:

#### 针对IOS

In [ ]:
        if system == 1:
            b1, a1 = signal.butter(2, [0.001, 0.03], 'bandpass')
            accyAll_After = signal.filtfilt(b1, a1, accyAll,padlen = 0)
            print('length:',len(accyAll_After))
            print('activity:',activity)
            print('len of activity:', len(activity))
            peaks = detect_peaks(accyAll_After)
            print('peaks:',peaks)
            print('len of peaks:', len(peaks))

##### 根据波峰在数组中的位置，在按时间存储动作标签的activity数组中寻找对应标签

In [ ]:
            for i in range(len(peaks)):
                location = int(peaks[i] / 60)
                if location == 0:
                    location = 1
                type = activity[location]
                if type == 1:
                    numA += 1
                if type == 2:
                    numB += 1
                if type == 3:
                    numC += 1
                if type == 4:
                    numD += 1
            count = [numA,numB,numC,numD]
            countStr =  ','.join(str(i) for i in count)

#### 针对安卓（原理同上）

In [ ]:
        if system == 2:
            b1, a1 = signal.butter(2, [0.025, 0.09], 'bandpass')
            accyAll_After = signal.filtfilt(b1, a1, accyAll, padlen=0)
            print('length:', len(accyAll_After))
            print('activity:', activity)
            print('len of activity:', len(activity))
            peaks = detect_peaks(accyAll_After)
            print('peaks:', peaks)
            print('len of peaks:', len(peaks))
            for i in range(len(peaks)):
                location = int(peaks[i] / 20)
                if location == 0:
                    location = 1
                type = activity[location]
                if type == 1:
                    numA += 1
                if type == 2:
                    numB += 1
                if type == 3:
                    numC += 1
                if type == 4:
                    numD += 1
            count = [numA,numB,numC,numD]
            countStr =  ','.join(str(i) for i in count)

    print(count)
    print(countStr)
    return countStr

### 动作识别函数

In [ ]:
@app.route('/',methods = ['post'])
def action():
    global accx
    global accy
    global accz
    global gryx
    global gryy
    global gryz
    global feature
    global system
    global label

#### 获取小程序发送来的数据，并将六轴数据和手机系统[system：1(ios)/2(安卓)]分别存储

In [ ]:
    data = request.get_json()

    accx1 = data['accx']
    accy1 = data['accy']
    accz1 = data['accz']
    gryx1 = data['gryx']
    gryy1 = data['gryy']
    gryz1 = data['gryz']
    system = data['system']
    print('system:',system)

#### 系统为IOS（system = 1），采样频率为20ms/组

In [ ]:
    if system == 1:

##### 将本次的 1s 数据（64个点）与上 1s 数据（64个点）进行拼接

In [ ]:
        accx = np.concatenate((accx,accx1))
        accy = np.concatenate((accy,accy1))
        accz = np.concatenate((accz,accz1))
        gryx = np.concatenate((gryx,gryx1))
        gryy = np.concatenate((gryy,gryy1))
        gryz = np.concatenate((gryz,gryz1))

#### 判断接收到的数据是否达到128个（作者设置为2.56s判断一次动作，而为了每次判断有50%重叠区，小程序端每 1.28s 发送一次数据（64个点）。因此该判断主要为了空过开始识别后的第一秒。）

In [ ]:
        if len(accx) >100:

            b, a = signal.butter(3, 0.1, 'lowpass')

##### 特征提取部分

In [ ]:
            accx = signal.filtfilt(b, a, accx)
            mean_x1 = mean(accx[:])
            std_x1 = std(accx[:])
            mad_x1 = mad(accx[:])
            max_x1 = max(accx[:])
            min_x1 = min(accx[:])

            sma_x1 = sma(accx[:])
            energy_x1 = energy(accx[:])
            skew_x1 = skew(accx[:])
            kurt_x1 = kurt(accx[:])
        # plt.plot(num, accx[0:1000])
        # plt.title('accx:')
        # python要用show展现出来图
        # plt.show()

            accy = signal.filtfilt(b, a, accy)
            mean_y1 = mean(accy[:])
            std_y1 = std(accy[:])
            mad_y1 = mad(accy[:])
            max_y1 = max(accy[:])
            min_y1 = min(accy[:])

            sma_y1 = sma(accy[:])
            energy_y1 = energy(accy[:])
            skew_y1 = skew(accy[:])
            kurt_y1 = kurt(accy[:])

        #   plt.plot(num, accy[0:1000])
        #  plt.title('accy:')
        # python要用show展现出来图
        # plt.show()

            accz = signal.filtfilt(b, a, accz)
            mean_z1 = mean(accz[:])
            std_z1 = std(accz[:])
            mad_z1 = mad(accz[:])
            max_z1 = max(accz[:])
            min_z1 = min(accz[:])

            sma_z1 = sma(accz[:])
            energy_z1 = energy(accz[:])
            skew_z1 = skew(accz[:])
            kurt_z1 = kurt(accz[:])
            #  plt.plot(num, accz[0:1000])
            #  plt.title('accz:')
            # python要用show展现出来图
            # plt.show()

            gryx = signal.filtfilt(b, a, gryx)
            mean_x2 = mean(gryx[:])
            std_x2 = std(gryx[:])
            mad_x2 = mad(gryx[:])
            max_x2 = max(gryx[:])
            min_x2 = min(gryx[:])

            sma_x2 = sma(gryx[:])
            energy_x2 = energy(gryx[:])
            skew_x2 = skew(gryx[:])
            kurt_x2 = kurt(gryx[:])
            # plt.plot(num, gryx[0:1000])
            # plt.title('gryx:')
            # python要用show展现出来图
            # plt.show()

            gryy = signal.filtfilt(b, a, gryy)
            mean_y2 = mean(gryy[:])
            std_y2 = std(gryy[:])
            mad_y2 = mad(gryy[:])
            max_y2 = max(gryy[:])
            min_y2 = min(gryy[:])

            sma_y2 = sma(gryy[:])
            energy_y2 = energy(gryy[:])
            skew_y2 = skew(gryy[:])
            kurt_y2 = kurt(gryy[:])
            # plt.plot(num, gryy[0:1000])
            # plt.title('gryy:',)
            # python要用show展现出来图
            # plt.show()

            gryz = signal.filtfilt(b, a, gryz)
            mean_z2 = mean(gryz[:])
            std_z2 = std(gryz[:])
            mad_z2 = mad(gryz[:])
            max_z2 = max(gryz[:])
            min_z2 = min(gryz[:])

            sma_z2 = sma(gryz[:])
            energy_z2 = energy(gryz[:])
            skew_z2 = skew(gryz[:])
            kurt_z2 = kurt(gryz[:])
            # plt.plot(num, gryz[0:1000])
            # plt.title('gryz:')
            # python要用show展现出来图
            # plt.show()

            feature = [mean_x1, std_x1, mad_x1, max_x1, min_x1,
                            sma_x1, energy_x1, skew_x1, kurt_x1,
                            mean_y1, std_y1, mad_y1, max_y1, min_y1,
                            sma_y1, energy_y1, skew_y1, kurt_y1,
                            mean_z1, std_z1, mad_z1, max_z1, min_z1,
                            sma_z1, energy_z1, skew_z1, kurt_z1,
                            mean_x2, std_x2, mad_x2, max_x2, min_x2,
                            sma_x2, energy_x2, skew_x2, kurt_x2,
                            mean_y2, std_y2, mad_y2, max_y2, min_y2,
                            sma_y2, energy_y2, skew_y2, kurt_y2,
                            mean_z2, std_z2, mad_z2, max_z2, min_z2,
                                                                                                                                                                           sma_z2, energy_z2, skew_z2, kurt_z2,
                            ]

##### 将特征集投入训练好的模型中（action为识别结果）

In [ ]:
            rf = joblib.load('model_RF_IOS_97.1_5Actions.model')
            feature = np.array(feature)
            feature = feature.reshape(1,-1)
            action = rf.predict(feature)

            print('action1:',action)
            action = str(action[0])
            print('action1:',action)
            feature = []

##### 如达到128个点，则将本次各数据的后一半保存，以方便与下一次的数据进行拼接，达到动作识别时有50%重叠区的目的

In [ ]:
            accx = accx[64:]
            accy = accx[64:]
            accz = accx[64:]
            gryx = accx[64:]
            gryy = accx[64:]
            gryz = accx[64:]

##### 如判断未达到128个点（即第一秒），则保留原数据不进行任何操作。

In [ ]:
        else :
            accx = accx[:]
            accy = accx[:]
            accz = accx[:]
            gryx = accx[:]
            gryy = accx[:]
            gryz = accx[:]
            action = '0'

#### 系统为安卓（system = 2） ，采样频率为60ms/组

In [ ]:
    if system == 2:

##### 将本次的 1s 数据（20个点）与上 1s 数据（20个点）进行拼接

In [ ]:
        accx = np.concatenate((accx, accx1))
        accy = np.concatenate((accy, accy1))
        accz = np.concatenate((accz, accz1))
        gryx = np.concatenate((gryx, gryx1))
        gryy = np.concatenate((gryy, gryy1))
        gryz = np.concatenate((gryz, gryz1))

##### 判断接收到的数据是否达到40个（作者设置为2.4s判断一次动作，而为了每次判断有50%重叠区，小程序端每 1.2s 发送一次数据（20个点）。因此该判断主要为了空过开始识别后的第一秒。）

In [ ]:
        if len(accx) > 30:

            b, a = signal.butter(3, 0.1, 'lowpass')

##### 特征提取部分

In [ ]:
            accx = signal.filtfilt(b, a, accx)
            mean_x1 = mean(accx[:])
            std_x1 = std(accx[:])
            mad_x1 = mad(accx[:])
            max_x1 = max(accx[:])
            min_x1 = min(accx[:])

            sma_x1 = sma(accx[:])
            energy_x1 = energy(accx[:])
            skew_x1 = skew(accx[:])
            kurt_x1 = kurt(accx[:])
            # plt.plot(num, accx[0:1000])
            # plt.title('accx:')
            # python要用show展现出来图
            # plt.show()

            accy = signal.filtfilt(b, a, accy)
            mean_y1 = mean(accy[:])
            std_y1 = std(accy[:])
            mad_y1 = mad(accy[:])
            max_y1 = max(accy[:])
            min_y1 = min(accy[:])

            sma_y1 = sma(accy[:])
            energy_y1 = energy(accy[:])
            skew_y1 = skew(accy[:])
            kurt_y1 = kurt(accy[:])

            #   plt.plot(num, accy[0:1000])
            #  plt.title('accy:')
            # python要用show展现出来图
            # plt.show()

            accz = signal.filtfilt(b, a, accz)
            mean_z1 = mean(accz[:])
            std_z1 = std(accz[:])
            mad_z1 = mad(accz[:])
            max_z1 = max(accz[:])
            min_z1 = min(accz[:])

            sma_z1 = sma(accz[:])
            energy_z1 = energy(accz[:])
            skew_z1 = skew(accz[:])
            kurt_z1 = kurt(accz[:])
            #  plt.plot(num, accz[0:1000])
            #  plt.title('accz:')
            # python要用show展现出来图
            # plt.show()

            gryx = signal.filtfilt(b, a, gryx)
            mean_x2 = mean(gryx[:])
            std_x2 = std(gryx[:])
            mad_x2 = mad(gryx[:])
            max_x2 = max(gryx[:])
            min_x2 = min(gryx[:])

            sma_x2 = sma(gryx[:])
            energy_x2 = energy(gryx[:])
            skew_x2 = skew(gryx[:])
            kurt_x2 = kurt(gryx[:])
            # plt.plot(num, gryx[0:1000])
            # plt.title('gryx:')
            # python要用show展现出来图
            # plt.show()

            gryy = signal.filtfilt(b, a, gryy)
            mean_y2 = mean(gryy[:])
            std_y2 = std(gryy[:])
            mad_y2 = mad(gryy[:])
            max_y2 = max(gryy[:])
            min_y2 = min(gryy[:])

            sma_y2 = sma(gryy[:])
            energy_y2 = energy(gryy[:])
            skew_y2 = skew(gryy[:])
            kurt_y2 = kurt(gryy[:])
            # plt.plot(num, gryy[0:1000])
            # plt.title('gryy:',)
            # python要用show展现出来图
            # plt.show()

            gryz = signal.filtfilt(b, a, gryz)
            mean_z2 = mean(gryz[:])
            std_z2 = std(gryz[:])
            mad_z2 = mad(gryz[:])
            max_z2 = max(gryz[:])
            min_z2 = min(gryz[:])

            sma_z2 = sma(gryz[:])
            energy_z2 = energy(gryz[:])
            skew_z2 = skew(gryz[:])
            kurt_z2 = kurt(gryz[:])
            # plt.plot(num, gryz[0:1000])
            # plt.title('gryz:')
            # python要用show展现出来图
            # plt.show()

            feature = [mean_x1, std_x1, mad_x1, max_x1, min_x1,
                       sma_x1, energy_x1, skew_x1, kurt_x1,
                       mean_y1, std_y1, mad_y1, max_y1, min_y1,
                       sma_y1, energy_y1, skew_y1, kurt_y1,
                       mean_z1, std_z1, mad_z1, max_z1, min_z1,
                       sma_z1, energy_z1, skew_z1, kurt_z1,
                       mean_x2, std_x2, mad_x2, max_x2, min_x2,
                       sma_x2, energy_x2, skew_x2, kurt_x2,
                       mean_y2, std_y2, mad_y2, max_y2, min_y2,
                       sma_y2, energy_y2, skew_y2, kurt_y2,
                       mean_z2, std_z2, mad_z2, max_z2, min_z2,
                       sma_z2, energy_z2, skew_z2, kurt_z2,
                       ]

##### 将特征集投入训练好的模型中（action为识别结果）

In [ ]:
            rf = joblib.load('model_RF_Android_97.0_5Actions.model')
            feature = np.array(feature)
            feature = feature.reshape(1, -1)
            action = rf.predict(feature)

            print('action1:', action)
            action = str(action[0])
            print('action1:', action)
            feature = []

##### 如达到40个点，则将本次各数据的后一半保存，以方便与下一次的数据进行拼接，达到动作识别时有50%重叠区的目的

In [ ]:
            accx = accx[20:]
            accy = accx[20:]
            accz = accx[20:]
            gryx = accx[20:]
            gryy = accx[20:]
            gryz = accx[20:]

##### 如判断未达到128个点（即第一秒），则保留原数据不进行任何操作。

In [ ]:
        else:
            accx = accx[:]
            accy = accx[:]
            accz = accx[:]
            gryx = accx[:]
            gryy = accx[:]
            gryz = accx[:]
            action = '0'

    return action

### 主函数（联系服务器）

In [ ]:
if __name__ == '__main__':
    context = (sys.path[0] + '/Nginx/1_www.inifyy.cn_bundle.crt', sys.path[0] + '/Nginx/2_www.inifyy.cn.key')
    app.run(debug=1, host='172.17.0.3', port=8000, ssl_context=context)
